In [1]:
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [2]:
train_feature = np.load('train_feature.npy')
train_label = np.load('train_label.npy')
test_feature = np.load('test_feature.npy')
test_label = np.load('test_label.npy')

In [4]:
train_feature = torch.from_numpy(train_feature)
train_feature = train_feature.float()
train_feature = Variable(train_feature)
train_feature = train_feature.cuda()

train_label = torch.from_numpy(train_label)
train_label = train_label.float()
train_label = Variable(train_label)
train_label = train_label.cuda()

test_feature = torch.from_numpy(test_feature)
test_feature = test_feature.float()
test_feature = Variable(test_feature)
test_feature = test_feature.cuda()

In [73]:
hidden_size = 18
num_layers = 1
learning_rate = 0.01
EPOCH = 600

In [74]:
class LSTMRG(nn.Module):
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
        super(LSTMRG, self).__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers, dropout=0.5)
        self.reg1 = nn.Linear(hidden_size, int(hidden_size/2))  # 拼接全连接层
        self.reg2 = nn.Linear(int(hidden_size/2), output_size)

    def forward(self, x):
        x, _ = self.rnn(x)
        s, b, h = x.shape
        x = x.view(s * b, h)
        x = self.reg1(x)
        x = F.tanh(x)
        x = self.reg2(x)
        x = x.view(s, b, -1)
        return x

In [75]:
for istation in range(81):
    for jinout in range(2):
        net = LSTMRG(input_size = train_feature.shape[-1], hidden_size = hidden_size, output_size=1, num_layers=num_layers)
        net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
        net.cuda()
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
        for epoch in range(EPOCH):
            net.train()
            out = net(train_feature[istation][jinout])
            loss = criterion(out, train_label[istation][jinout])
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if (epoch + 1) % EPOCH == 0:
                net.eval()
                pred_label = net(test_feature[istation][jinout])
                pred_label = pred_label.data.cpu().numpy()
                mse = np.mean((pred_label - test_label[istation][jinout]) ** 2)
                print('\tTrain Epoch: %d, train_loss: %.7f, test_loss: %.7f' % (epoch + 1, loss, mse))
        torch.save(net.state_dict(), 'model/station-' + str(istation).zfill(2) + '-' + str(jinout) + '.pth')

	Train Epoch: 600, train_loss: 0.0013558, test_loss: 0.0012953
	Train Epoch: 600, train_loss: 0.0017885, test_loss: 0.0024750
	Train Epoch: 600, train_loss: 0.0004533, test_loss: 0.0005581
	Train Epoch: 600, train_loss: 0.0018720, test_loss: 0.0020371
	Train Epoch: 600, train_loss: 0.0013014, test_loss: 0.0015776
	Train Epoch: 600, train_loss: 0.0006063, test_loss: 0.0005727
	Train Epoch: 600, train_loss: 0.0008405, test_loss: 0.0007503
	Train Epoch: 600, train_loss: 0.0015026, test_loss: 0.0016506
	Train Epoch: 600, train_loss: 0.0006950, test_loss: 0.0008599
	Train Epoch: 600, train_loss: 0.0003697, test_loss: 0.0004642
	Train Epoch: 600, train_loss: 0.0006845, test_loss: 0.0009691
	Train Epoch: 600, train_loss: 0.0004831, test_loss: 0.0004383
	Train Epoch: 600, train_loss: 0.0014928, test_loss: 0.0022309
	Train Epoch: 600, train_loss: 0.0030422, test_loss: 0.0030780
	Train Epoch: 600, train_loss: 0.0026175, test_loss: 0.0032024
	Train Epoch: 600, train_loss: 0.0015766, test_loss: 0.